# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [11]:
import csv
import urllib.request
import pandas as pd
import tweepy
import os
import os.path
import requests
import json
import time

In [12]:
cwd = os.getcwd() 
base_path = os.getcwd()
cwd

'/home/workspace'

In [13]:
def file_exists(base_p, file_name):
    return(os.path.exists(os.path.join(base_p, file_name)))

In [14]:
if (not(file_exists(base_path,"twitter-archive-enhanced.csv"))):
    url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'
    file_to_read = urllib.request.urlopen(url)
    df_ta = pd.read_csv(file_to_read)
    df_ta.to_csv(tw_arch_file)
else:
    file_to_read = os.path.join(base_path, "twitter-archive-enhanced.csv")
    df_ta = pd.read_csv(file_to_read)
df_ta.head(3)

,Unnamed: 0,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [15]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = urllib.request.urlopen(url)
df_ipr = pd.read_csv(response, sep = '\t')
df_ipr.head(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True


get all tweet_ids

In [16]:
tweet_ids = df_ta['tweet_id'].astype(str)
tweet_ids.shape[0]

2356

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [17]:
path_to_bearer_token = os.path.join(base_path, "bearer_token.txt")
with open(path_to_bearer_token, 'r') as f:    
    bearer_token = f.read()

In [57]:
def get_tweat_url(tw_id):
    twitter_url = "https://api.twitter.com/2/tweets/" + str(tw_id)
    query_params = {#'id': tw_id,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,text',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (twitter_url, query_params)

def get_tweats_url(tw_ids):
    twitter_url = "https://api.twitter.com/2/tweets"
    query_params = {'ids': tw_ids,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,text',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (twitter_url, query_params)

def connect_to_twitter(url, headers, params, next_token = None):
    params['next_token'] = next_token
    resp = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(resp.status_code))
    if resp.status_code != 200:
        raise Exception(resp.status_code, resp.text)
    return (resp.json())
  
    
def get_ids_string(start, quantity, tw_ids_series): #format sting with ids for pulling
    result = ''
    for i in range(start, start + quantity):
        if i in tw_ids_series.index:
            if len(result) > 0:
                result = result +','+ tw_ids_series[i]
            else:
                result = tw_ids_series[i]
        else:
            i-=1
            break
    return (result, i-start+1)
def add_timestamp(my_series):
    print('type_my_ser:', type(my_series) )
    cur_time = time.time()
    with open(os.path.join(base_path, 'rqsts.csv'),'a') as file:
        file.write(str(cur_time))
        file.write('\n')
    my_series = my_series.values.tolist()
    my_series.append(cur_time)
    my_series = pd.Series(my_series)
    return(my_series)
    
def make_request(tweets_ids, time_window = 1700, requests_limit = 50):
    if (file_exists(base_path, 'rqsts.csv')):
        rqsts = pd.read_csv(os.path.join(base_path, 'rqsts.csv'), names = ['timestamp'])
        rqsts_s = rqsts['timestamp']
    else:
        rqsts_s = pd.Series([])
    current_time = time.time()
    rqsts_f = rqsts_s[rqsts_s > (current_time - time_window)]
    if (rqsts_f.shape[0] < requests_limit):
        #do request
        url = get_tweats_url(tweets_ids)
        headers = {"Authorization": "Bearer " + bearer_token}
        rqsts_f = rqsts_f.values.tolist()
        rqsts_f.append(current_time)
        rqsts_f = pd.Series(rqsts_f)
        rqsts_f.to_csv(os.path.join(base_path, 'rqsts.csv'))
        json_resp = connect_to_twitter(url[0], headers, url[1])
        print("request completed")
        return(json_resp, True)
    else:
        print('limit of request exceeded: requests= ', rqsts_f.shape[0], 'limit=', requests_limit)
        return(0, False)
    
#read_JSON RABOTAET NEPRVILNO https://github.com/pandas-dev/pandas/issues/20608# (pri chtenii)
def pull_all_data(ids_series, batch_size = 5, time_window = 950, req_limit = 50, sleep_time = 10):
    if (file_exists(base_path, 'tweet_json.txt')):
        #workaround the insidious bug in read_json method of pandas: "https://github.com/pandas-dev/pandas/issues/20608#"
        #some random id's in my dataframe became  a little bit different tnan they should be...
        #I spend one hour to understand that my code is fine, it's pandas bug :-)
        tweet_json_df = pd.read_json(os.path.join(base_path, 'tweet_json.txt'), lines=True, orient='index', convert_axes=False, dtype={}).transpose()
        last_id = tweet_json_df.iloc[-1:]['id'].values[0] # the id of the last tweet in the output file (single value)
        idx = (df_ta.index[df_ta['tweet_id'].astype(str) == last_id])
        start_line = (idx[0].item()) #number of string to start, converted to basic python type
    else:
        start_line = 0
    count = 0
    for current_line in range(start_line, ids_series.shape[0], batch_size):
        ids = get_ids_string(current_line, batch_size, ids_series)
        rsp = ('', False)
        while (rsp[1]!= True):
            time.sleep(1)
            rsp = make_request(ids[0],time_window, req_limit)
            if (rsp[1]!= True): #we exceed the limit
                print('waiting ', sleep_time, ' seconds' )
                time.sleep(sleep_time)
        if (not((rsp[0].get('data')) is None)):
            missed_id_flag = False
            for object_number in range(0,len(rsp[0].get('data'))): #check, maybe ids[1] +1
                if (len(rsp[0].get('data')) < batch_size):
                    missed_id_flag = True
                with open('tweet_json.txt', 'a') as out_f:
                    json.dump(rsp[0].get('data')[object_number],out_f)
                    out_f.write('\n')
                    count +=1
            if (missed_id_flag):
                print('Missed id is here ------------>')
    print('records completed =', count)

In [58]:
pull_all_data(df_ta['tweet_id'].astype(str).head(200), batch_size = 20, time_window = 905, req_limit = 50, sleep_time = 10)

('892420643555336193,892177421306343426,891815181378084864,891689557279858688,891327558926688256,891087950875897856,890971913173991426,890729181411237888,890609185150312448,890240255349198849,890006608113172480,889880896479866881,889665388333682689,889638837579907072,889531135344209921,889278841981685760,888917238123831296,888804989199671297,888554962724278272,888202515573088257', 20)
Endpoint Response Code: 200
request completed
Missed id is here ------------>
('888078434458587136,887705289381826560,887517139158093824,887473957103951883,887343217045368832,887101392804085760,886983233522544640,886736880519319552,886680336477933568,886366144734445568,886267009285017600,886258384151887873,886054160059072513,885984800019947520,885528943205470208,885518971528720385,885311592912609280,885167619883638784,884925521741709313,884876753390489601', 20)
Endpoint Response Code: 200
request completed
('884562892145688576,884441805382717440,884247878851493888,884162670584377345,883838122936631299,883

In [59]:
# find all missed id's and try to get it again
tweet_json_df = pd.read_json(os.path.join(base_path, 'tweet_json.txt'), lines=True, orient='index', convert_axes=False, dtype={}).transpose()
df2 = tweet_json_df[['id','conversation_id']]
df1 = df_ta[['tweet_id','timestamp']].head(200)
df1['id'] = df_ta['tweet_id'].astype(str)
df = pd.merge(df1, df2, on ='id', how ="left")[pd.merge(df1, df2, on ='id', how ="left")['conversation_id'].isna()]
missed_ids = df['tweet_id'].astype(str).str.cat(sep=',')
print(missed_ids)


888202515573088257,873697596434513921,872668790621863937,872261713294495745,869988702071779329,866816280283807744,861769973181624320,856602993587888130,856330835276025856


In [61]:
rsp = make_request(missed_ids,905, 50)
if (not((rsp[0].get('data')) is None)):
    for object_number in range(0,len(rsp[0].get('data'))): 
        if (len(rsp[0].get('data')) < 9):
            print('HERE------------>')
        with open('bbb.txt', 'a') as out_f:
            json.dump(rsp[0].get('data')[object_number],out_f)
            out_f.write('\n')
else:
    print('data = null')
               

Endpoint Response Code: 200
request completed
data = null


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization